<div style="text-align:center">
<h1> CSCI 3155 Principles of Programming Languages </h1>
<h2> Spring 2025</h2>
</div>

### Today's Lecture
+ Big-step Operational Semantics
  * Handling error
  * Implementing semantics as interpreter
+ Higher-order functions

## Arithmetic Expressions

Arithmetic expressions are generated by the grammar we saw before.

$$\begin{array}{rcll}
\textbf{Expr} & \rightarrow & Const(\textbf{Double}) \\
& |  & Ident(\textbf{Identifier}) \\
& | & Plus( \textbf{Expr}, \textbf{Expr})  \\
& | & Minus( \textbf{Expr}, \textbf{Expr}) \\
& | & Mult(\textbf{Expr}, \textbf{Expr}) \\
& | & Div(\textbf{Expr}, \textbf{Expr}) \\
& | & Log(\textbf{Expr}) \\
& | & Exp(\textbf{Expr}) \\
& | & Sine(\textbf{Expr}) \\
& | & Cosine(\textbf{Expr}) \\\\
\textbf{Double} & \rightarrow & \text{all double precision numbers in Scala}\\
\textbf{Identifier} & \rightarrow & [a-zA-Z][a-z\ A-Z\ 0-9\ \_]* & \text{Note: All strings that begin with letters}\\
&&& \text{a-z or A-Z and subsequently can contain a-z, A-Z, 0-9 or \_ chars}
\end{array}$$

In [1]:
sealed trait Expr
case class Const(c:Double) extends Expr
case class Ident(v:String) extends Expr
case class Plus(e1:Expr, e2: Expr) extends Expr
case class Minus(e1:Expr, e2: Expr) extends Expr
case class Mult(e1:Expr, e2: Expr) extends Expr
case class Div(e1:Expr, e2: Expr) extends Expr
case class Sine(e:Expr) extends Expr
case class Cosine(e:Expr) extends Expr
case class Exp(e:Expr) extends Expr

defined trait Expr
defined class Const
defined class Ident
defined class Plus
defined class Minus
defined class Mult
defined class Div
defined class Sine
defined class Cosine
defined class Exp

## Big Step Semantics
$\newcommand{\eval}[2]{\mathbf{eval}(#1) = #2}$

Inference rules are always written like this.

$$\begin{array}{c}
\text{premises that must hold} \\
\hline
\text{conclusions that can be drawn} \\
\end{array}$$

It must be read as __assuming premises must hold, then conclusion must hold__.

As an example, let us see a rule for `Log`:

$$\begin{array}{c}
\eval(\texttt{e},\sigma) = c \\
\hline 
\eval(\texttt{Log(e)}, \sigma) = \log(c) \\
\end{array}\ \text{(Log)}$$

The rule says:
- Assume: "under the environment $\sigma$, some expression `e` evaluates to $c$"
- Conclude: "under the environment $\sigma$, the expression `Log(e)` evalutes to $\log(c)$"

#### Inference Rules


$$ \begin{array}{c}
\\
\hline
\eval\left(\texttt{Const(f)}, \sigma\right) = f \\
\end{array}\ \text{(Const)} $$


$$ \begin{array}{c}
\\
\hline
\eval(\texttt{Ident(s)}, \sigma) = \sigma(s)\\
\end{array}\ \text{(Variable)} $$

### Environments

An environment is a partial function from names of identifiers to their values.
- Let $Domain(\sigma)$ be the set of all variables defined in an environment.
- Let $\sigma(s)$ be the value mapped to by identifier $s$ if $s \in Domain(\sigma)$.


### Inference Rules (Continued)


$$\begin{array}{c}
s \in \text{Domain}(\sigma)\\
\hline
\eval(\texttt{Ident(s)},\sigma) = \sigma(s) \\
\end{array}\ (\text{Ident})$$

Can you read this rule aloud for us?

- Premise: the identifier $s$ belongs to $\text{Domain}(\sigma)$
- Conclusion: $\eval(\texttt{Ident(s)},\sigma) = \sigma(s)$


#### Plus

$$\begin{array}{c}
\eval(\texttt{e1}, \sigma) = c_1,\ \eval(\texttt{e2}, \sigma) =  c_2\\
\hline
\eval\left( \texttt{Plus(e1,e2)}, \sigma\right)  =  (c_1 + c_2) \\
\end{array} (\text{Plus}) $$


#### Minus

$$\begin{array}{c}
\eval(\texttt{e1}, \sigma) = c_1,\ \eval(\texttt{e2}, \sigma) =  c_2 \\
\hline
\eval\left(\texttt{Minus(e1, e2)}, \sigma\right) = (c_1 - c_2 ) \\
\end{array} (\text{Minus}) $$

#### Multiplication

$$\begin{array}{c}
\eval(\texttt{e1}, \sigma) = c_1,\ \eval(\texttt{e2}, \sigma) =  c_2 \\
\hline
\eval\left(\texttt{Mult(e1, e2)}, \sigma\right) = (c_1 \times c_2) \\
\end{array} (\text{Mult}) $$

#### Division

$$\begin{array}{c}
\eval(\texttt{e1}, \sigma) = c_1,\ \eval(\texttt{e2}, \sigma) =  c_2,\ \color{red}{c_2 \not= 0}\\
\hline
\eval\left( \texttt{Div(e1, e2)}, \sigma \right) = (\frac{c_1}{c_2} ) \\
\end{array} (\text{Div}) $$




#### Other Rules

$$\begin{array}{c}
\eval\left( \texttt{e}, \sigma\right) = c,\ \color{red}{c > 0} \\
\hline 
\eval(\texttt{Log(e)}, \sigma) = \log(c) \\
\end{array}\ \text{(Log)}$$

#### Rule Templates


$$f_{\texttt{Exp}}(x) = e^x,\ f_{\texttt{Sine}}(x) = \sin(x),\ f_{\texttt{Cosine}}(x) = \cos(x)$$

$$\begin{array}{c}
\eval(\texttt{e}, \sigma) = c,\ T \in \{ \texttt{Exp}, \texttt{Sine}, \texttt{Cosine} \} \\
\hline
\eval\left( \texttt{T(e)}, \sigma \right) =  f_{\texttt{T}}(c)
\end{array} (\text{InBuilt-Function-Application})$$



In [4]:
// Interpreter for our very simple arithmetic language
// binOpHelper is a "higher-order" function.
def binOpHelper(env:Map[String,Double], e1:Expr, e2:Expr, f: (Double,Double) => Double) : Double = {
    val c1 = evalExpr(env, e1)
    val c2 = evalExpr(env, e2)
    f(c1,c2)
}

def unOpHelper(env:Map[String,Double], e1:Expr, f: (Double) => Double) : Double = {
    val c1 = evalExpr(env, e1)
    f(c1)
}

def evalExpr(env: Map[String,Double], e:Expr) : Double = {
    e match {
        case Const(c) => c
        case Ident(v) => env(v)
        case Plus(e1,e2) => {
            def plus(x:Double,y:Double) = x+y
            binOpHelper(env, e1,e2, plus)
        }
        case Minus (e1,e2) => {
            def minus(x:Double,y:Double) = x-y
            binOpHelper(env, e1, e2 ,minus)
        }
        case Mult(e1,e2) => {
            def mult(x:Double,y:Double) = x*y
            binOpHelper(env, e1, e2 ,mult)
        }
        case Div(e1,e2) => {
            def div(c1:Double,c2:Double) = {
                require(c2!=0)
                c1/c2 
            }
            binOpHelper(env, e1, e2, div)
        }
        case Sine(e1) => {
            def mysine(c1:Double) = math.sin(c1)
            unOpHelper(env, e1, mysine)
        }
        case Cosine(e1) => {
            val c1 = evalExpr(env, e1)
            math.cos(c1)
        }
        case Exp(e1) => {
            val v1 = evalExpr(env, e1)
            math.exp(v1)
        }
    }
}

defined function binOpHelper
defined function unOpHelper
defined function evalExpr

(2-2) * (2x^3 + 2*sin^2(x) + 4)

## Short circuiting

$$\begin{array}{c}
\eval(\texttt{e1}, \sigma) =  0.0,\\
\hline
\eval(\texttt{Mult(e1, e2)}, \sigma) = 0.0 \end{array} (\text{Shortciruit-Mult})$$


## Handling Error
$\newcommand{\eval}[2]{\mathbf{eval}(#1) = #2}$

$$\begin{array}{c}
\eval(\texttt{e1}, \sigma) = c_1,\ \eval(\texttt{e2}, \sigma) =  c_2,\ \color{red}{c_1\in \mathbb{R}, c_2 \in \mathbb{R}}\\
\hline
\eval\left( \texttt{Plus(e1, e2)}, \sigma\right)  =  (c_1 + c_2 ) \\
\end{array} (\text{Plus}) $$

$$\begin{array}{c}
\eval(\texttt{e1}, \sigma) = c_1,\ \eval(\texttt{e2}, \sigma) =  c_2,\ \color{red}{c_1\in \mathbb{R}, c_2 \in \mathbb{R}} \\
\hline
\eval\left(\texttt{Minus(e1,e2)}, \sigma\right) = (c_1 - c_2) \\
\end{array} (\text{Minus}) $$

$$\begin{array}{c}
\eval(\texttt{e}, \sigma) = c,\ T \in \{ \texttt{Exp}, \texttt{Sine}, \texttt{Cosine} \}, \color{red}{c \in \mathbb{R}}\\
\hline
\eval\left( \texttt{T(e)}, \sigma \right) =  f_{\texttt{T}}(c)
\end{array} (\text{InBuilt-Function-Application})$$

How about __error__? First we will summarize all situations that can produce a value __error__.

$$ \begin{array}{c}
 s \not\in \text{Domain}(\sigma) \\
 \hline
 \eval(\texttt{Ident(s)}, \sigma) = \mathbf{error} \\
 \end{array} (\text{Ident-ERROR}) $$

Other situations are also easy to see:

$$\begin{array}{c}
\eval(\texttt{e1}, \sigma) =  c_1,\ \eval(\texttt{e2}, \sigma) = c_2,\ {c_1 \in \mathbb{R}, c_2 \in \mathbb{R}},\ c_2 = 0\\
\hline
\eval\left(\texttt{Div(e1, e2)}, \sigma\right) = \mathbf{error} \\
\end{array} (\text{Div-ERROR}) $$

Another situation that comes to mind is `Log`.


$$\begin{array}{c}
\eval(\texttt{e}, \sigma) = c,\ {c \in \mathbb{R}},\ {c \leq 0} \\
\hline 
\eval(\texttt{Log(e)}, \sigma) =  \mathbf{error} \\
\end{array}\ \text{(Log-ERROR)}$$

Now we need to write rules that say that once any child of an expression evaluates to an __error__ the expression itself evaluates to an error. This is very cumbersome to do in its fullest exquisite detail. Therefore, we will say so using appropriate notation to help us.

Let us define the set of a __subterms__ of a given term inductively.
- For expressions $e$ of the form `Plus(e1, e2)`, `Minus(e1, e2)`, and
`Mult(e1, e2)` and `Div(e1, e2)`: 
$$ \text{subterm(e)} = \{ \texttt{e1}, \texttt{e2} \} \cup  \text{subterm}(\texttt{e1}) \cup \text{subterm}(\texttt{e2})$$
- For expressions $e$ of the form `T(e1)` where `T` can be `Log, Sine, Cosine, Exp`:
$$\text{subterm}(e) = \{ \texttt{e1} \} \cup \text{subterm}(\texttt{e1})$$

__Example:__ Using the definintion, we can show that 
- subterm(`Plus(Const(1.0) , Mult(Const(2.0), Ident("y"))`) is the set { `Const(1.0)`, `Mult(Const(2.0), Ident("y"))`, `Const(2.0)`, `Ident("y")` }


Now we can write a single rule to deal with __error__:

$$\begin{array}{c}
\eval(\texttt{e1}, \sigma) = \mathbf{error},\ \texttt{e1} \in \text{subterm}(e) \\
\hline
\eval( \texttt{e}, \sigma) =  \mathbf{error} \end{array} (\text{Subterm-ERROR})$$

Let's interpret this rule:
- Premise: under environment $\sigma$ the expression `e1` evaluates to __error__ and `e1` is a subterm of `e`.
- Conclusion: under environment $\sigma$ the expression `e` evalutes to __error__.

The bigstep semantics now clarify how some ambiguous function situations are to be handled.  

In [5]:
sealed trait Value
case class Number(n:Double) extends Value
case object Error extends Value

defined trait Value
defined class Number
defined object Error

In [6]:
// Interpreter for our very simple arithmetic language
// binOpHelper is a "higher-order" function.
def binOpHelper(env:Map[String,Double], e1:Expr, e2:Expr, f: (Double,Double) => Double) : Value = {
    val c1 = evalExpr(env, e1)
    c1 match {
        case Error => Error
        case Number(n1) => {
            val c2 = evalExpr(env,e2)
            c2 match {
                case Error => Error
                case Number(n2) => Number(f(n1,n2))
            }
        }
    }
}

def unOpHelper(env:Map[String,Double], e1:Expr, f: (Double) => Double) : Value = {
    val c1 = evalExpr(env, e1)
    c1 match {
        case Error => Error
        case Number(n1) => Number(f(n1))
    }
}

def evalExpr(env: Map[String,Double], e:Expr) : Value = {
    e match {
        case Const(c) => Number(c)
        case Ident(v) => {
            try{
                Number(env(v))
            } catch(e:Exception) {
                Error
            }
        }
        case Plus(e1,e2) => {
            def plus(x:Double,y:Double) = x+y
            binOpHelper(env, e1,e2, plus)
        }
        case Minus (e1,e2) => {
            def minus(x:Double,y:Double) = x-y
            binOpHelper(env, e1, e2 ,minus)
        }
        case Mult(e1,e2) => {
            def mult(x:Double,y:Double) = x*y
            binOpHelper(env, e1, e2 ,mult)
        }
        case Div(e1,e2) => {
            def div(c1:Double,c2:Double) = {
                require(c2!=0)
                c1/c2 
            }
            binOpHelper(env, e1, e2, div)
        }
        case Sine(e1) => {
            def mysine(c1:Double) = math.sin(c1)
            unOpHelper(env, e1, mysine)
        }
        /*
        case Cosine(e1) => {
            math.cos(c1)
        }
        case Exp(e1) => {
            val v1 = evalExpr(env, e1)
            math.exp(v1)
        }
        */
    }
}

cmd6.sc:31: type mismatch;
 found   : cmd6.this.cmd1.Expr
 required: Exception
            } catch(e:Exception) {
                    ^
cmd6.sc:31: This catches all Throwables. If this is really intended, use `case _ : Throwable` to clear this warning.
            } catch(e:Exception) {
                                 ^
Compilation Failed